In [ ]:
#default_exp sync

# Synchronize and diff

> Propagating small changes in the library back to notebooks

The library is primarily developed in notebooks so any big changes should be made there. But sometimes, it's easier to fix small bugs or typos in the modules directly. `nbdev_update_lib` is the function that will propagate those changes back to the corresponding notebooks. Note that you can't create new cells or reorder cells with that functionality, so your corrections should remain limited.

In [ ]:
#export
from nbdev.read import *
from nbdev.export import *

from nbdev.imports import *
from fastcore.script import *

import nbformat,ast
from nbformat.sign import NotebookNotary

In [ ]:
#export
_re_import = re.compile("from\s+\S+\s+import\s+\S")

def _to_absolute(code, libname):
    if not _re_import.search(code): return code
    res = update_import(code, ast.parse(code).body, libname, absolute_import)
    if not res: return code
    return ''.join(res)

def _update_lib(nbname, nb_locs):
    libname = Config().lib_name
    nbtxt = Path(nbname).read_text()
    nb = nbformat.reads(nbtxt, as_version=4)
    for nbn,cellid,code in nb_locs:
        assert nbn==nbname,(nbn,nbname)
        code = code[code.find("\n")+1:]
        nb.cells[int(cellid)].source = _to_absolute(code, libname)

    NotebookNotary().sign(nb)
    nbformat.write(nb, 'tmp/tmp.ipynb', version=4)

In [ ]:
#export
@call_parse
def nbdev_update_lib(fname:Param("A python module name to convert", str)):
    "Propagates any change in the modules matching `fname` to the notebooks that created them"
    if os.environ.get('IN_TEST',0): return
    fname = Path(fname)
    code = fname.read_text()
    split_str = "\n#nbdev_cell "
    code_cells = code.split(split_str)[1:]
    locs = L(s.splitlines()[0].split()+[s.strip("\n")]
             for s in code_cells if not s.startswith('auto '))
    for nbname,nb_locs in groupby(locs, itemgetter(0)).items(): _update_lib(nbname, nb_locs)

In [ ]:
# nbdev_update_lib("../nbdev/export.py")

## Export -

In [ ]:
from nbdev.export import nbdev_build_lib
nbdev_build_lib()